In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pathlib import Path

In [8]:
# Для начала загрузим данные:

df = pd.read_excel("data_clean.xlsx")

print("Первые строки датасета:")
print()
print(df.head(), "\n")

print("Информация о датасете:")
print()
print(df.info(), "\n")

Первые строки датасета:

              author        author_type           district  \
0             Алькор          developer      Кронштадтский   
1  Сити-Недвижимость  real_estate_agent      Петроградский   
2             Алькор          developer      Кронштадтский   
3       House Estate  real_estate_agent      Петроградский   
4             Bonava          developer  Красногвардейский   

                street house_number               metro   residential_complex  \
0  дорога Цитадельская          NaN             Беговая  Кронфорт.Центральный   
1  Петровский проспект         24к3  Крестовский остров    Петровская ривьера   
2  дорога Цитадельская          NaN             Беговая  Кронфорт.Центральный   
3           Зверинская          7-9          Спортивная                   NaN   
4       Магнитогорская          NaN      Новочеркасская   Magnifika Residence   

   total_meters  living_meters  kitchen_meters  rooms_count  floor  \
0         36.25           14.4            12.

In [10]:
# Очистим датасет от пропусков - по возможности заполним их медианными значениями:

df = df.dropna(subset=["price", "total_meters"]).copy()

num_cols_median = [
    "living_meters",
    "kitchen_meters",
    "house_floors_total",
    "construction_age",
    "year_of_construction",
]
for col in num_cols_median:
    if df[col].isna().any():
        df[col] = df[col].fillna(df[col].median())

cat_cols_unknown = [
    "author_type",
    "house_material_type",
    "finish_type",
    "metro",
    "district",
]
for col in cat_cols_unknown:
    df[col] = df[col].fillna("Unknown")

In [12]:
# Добавим в датасет новые признаки для будущей проверки гипотез - бинарная central (центральный район СпБ), бинарная middle_floor ("средний этаж") - 
# выше первого, но ниже последнего, бинарная old_house - старый жилой фонд (дома с возрастом > 100 лет), бинарная new_build - новострой 
# (отрицательный год постройки - construction_age):

central_districts = [
    "Адмиралтейский",
    "Центральный",
    "Петроградский",
    "Василеостровский",
]
df["central"] = df["district"].isin(central_districts).astype(int)

df["middle_floor"] = (
    (df["floor"] > 1) & (df["floor"] < df["house_floors_total"])
).astype(int)

current_year = 2025
df["old_house"] = ((current_year - df["year_of_construction"]) >= 100).astype(int)

df["new_build"] = (df["construction_age"] < 0).astype(int)

# Также настроим "взаимодействия" для проверки гипотезы №3 “Историческая и ценовая значимость памятников архитектуры”:

df["central_old"] = df["central"] * df["old_house"]
df["outsk_new"] = (1 - df["central"]) * df["new_build"]

# И введем логарифм цены, так будет удобнее интерпретировать проценты, по идее:

df["log_price"] = np.log(df["price"])

# Наконец соберем финальный набор переменных:

model_cols = [
    "log_price",
    "total_meters",
    "floor",
    "house_floors_total",
    "central",
    "middle_floor",
    "old_house",
    "central_old",
    "outsk_new",
    "living_meters",
    "kitchen_meters",
]
df_model = df[model_cols].dropna().copy()

print("Новые признаки:")
print()
print(df_model.head(), "\n")
print()
print(f"Итоговый размер выборки: {df_model.shape[0]} наблюдений\n")

Новые признаки:

   log_price  total_meters  floor  house_floors_total  central  middle_floor  \
0  15.925571         36.25      3                   4        0             1   
1  17.312018         70.10      5                   9        1             1   
2  16.436205         59.92      3                   4        0             1   
3  17.073607         67.20      3                   5        1             1   
4  16.929462         56.60      1                  14        0             0   

   old_house  central_old  outsk_new  living_meters  kitchen_meters  
0          0            0          1           14.4            12.2  
1          0            0          0           35.0            21.3  
2          0            0          1           33.0            16.6  
3          1            1          0           15.0            31.0  
4          0            0          0           15.9            22.0   


Итоговый размер выборки: 1379 наблюдений



In [14]:
# Теперь перейдем к построению регрессионной модели:

# Базовая OLS-модель с робастными SE:

y = df_model["log_price"]
X = df_model[
    [
        "total_meters",
        "floor",
        "house_floors_total",
        "central",
        "middle_floor",
        "old_house",
        "central_old",
        "outsk_new",
    ]
]
X = sm.add_constant(X)

ols = sm.OLS(y, X).fit(cov_type="HC1")  # тут применим робастные (White) SE

print(ols.summary(), "\n")

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.822
Method:                 Least Squares   F-statistic:                     331.7
Date:                Sat, 17 May 2025   Prob (F-statistic):          5.06e-314
Time:                        19:50:15   Log-Likelihood:                -140.46
No. Observations:                1379   AIC:                             298.9
Df Residuals:                    1370   BIC:                             346.0
Df Model:                           8                                         
Covariance Type:                  HC1                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 15.1735      0

In [16]:
# Видим, что значения R^2 и Adj. R^2 находятся на достаточно высоком уровне, поэтому,
# можем остановиться на тестировании только этого варианта модели и перейти к диагностике мультиколлинеарности через VIF:

vif = pd.DataFrame(
    {
        "feature": X.columns,
        "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
    }
)
print("VIF по регрессорам:")
print()
print(vif, "\n")

VIF по регрессорам:

              feature        VIF
0               const  12.621893
1        total_meters   1.489426
2               floor   1.394679
3  house_floors_total   1.510905
4             central   1.679798
5        middle_floor   1.150161
6           old_house   7.197952
7         central_old   7.358904
8           outsk_new   1.156336 



In [18]:
# Заметим, что VIF < 10 лдя всех регрессоров, а значит - сильной мульитиколлинеарности у нас не наблюдается,
# можем перейти к проведению тестов на эндогенность площади при помощи DWH-теста в 2 этапа:

# Этап 1 - total_meters ~ инструменты (= кухня + жилая + те же X-кроме-площади):

instr_cols = (
    ["living_meters", "kitchen_meters"]
    + [
        "floor",
        "house_floors_total",
        "central",
        "middle_floor",
        "old_house",
        "central_old",
        "outsk_new",
    ]
)

Z = sm.add_constant(df_model[instr_cols])

first_stage = sm.OLS(df_model["total_meters"], Z).fit()

print("Первая стадия (total_meters):")
print()
print(first_stage.summary(), "\n")

# Этап 2 - добавим остатки 1-й стадии в исходное уравнение:

df_model["fs_resid"] = first_stage.resid
X_aug = sm.add_constant(
    df_model[
        [
            "total_meters",
            "floor",
            "house_floors_total",
            "central",
            "middle_floor",
            "old_house",
            "central_old",
            "outsk_new",
            "fs_resid",
        ]
    ]
)

dwh = sm.OLS(df_model["log_price"], X_aug).fit(cov_type="HC1")

print("DWH — значимость fs_resid:")
print()
print(dwh.summary().tables[1], "\n")

Первая стадия (total_meters):

                            OLS Regression Results                            
Dep. Variable:           total_meters   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.835
Method:                 Least Squares   F-statistic:                     778.2
Date:                Sat, 17 May 2025   Prob (F-statistic):               0.00
Time:                        19:56:28   Log-Likelihood:                -5102.5
No. Observations:                1379   AIC:                         1.022e+04
Df Residuals:                    1369   BIC:                         1.028e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const

In [20]:
# По результатам DWH-теста можем заметить, что  коэффициент у fs_resid оказался статистически незначим (p = 0.44),
# а значит, что признаков эндогенности площади у нас нет, можем перейти к следующему этапу - проведению двухшагового МНК 2SLS, 
# также пойдем в 2 этапа:

# Этап 1 - возьмем предсказанные total_meters с 1-й стадии предыдущего шага:

df_model["total_meters_hat"] = first_stage.fittedvalues

# Этап 2 - log_price ~ total_meters_hat + прочие X, используем IV-оценки:

X_iv = sm.add_constant(
    df_model[
        [
            "total_meters_hat",
            "floor",
            "house_floors_total",
            "central",
            "middle_floor",
            "old_house",
            "central_old",
            "outsk_new",
        ]
    ]
)

iv = sm.OLS(df_model["log_price"], X_iv).fit(cov_type="HC1")

print("IV-/2SLS-оценка (робастные SE):")
print()
print(iv.summary(), "\n")
print()
print()
print(f"F-статистика 1-й стадии (Staiger-Stock): {first_stage.fvalue:.2f}\n")

IV-/2SLS-оценка (робастные SE):

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     226.0
Date:                Sat, 17 May 2025   Prob (F-statistic):          5.10e-244
Time:                        20:02:23   Log-Likelihood:                -394.75
No. Observations:                1379   AIC:                             807.5
Df Residuals:                    1370   BIC:                             854.6
Df Model:                           8                                         
Covariance Type:                  HC1                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
con

In [24]:
# Видим, что IV-оценка дала практически тот же эффект площади, как и OLS (0.017 в лог-шкале), а Hausman-статистика не отвергает OLS,
# F = 778 >= 10 по Staiger-Stock-Rule, что означает отсуствие у нас слабых инструментов, можем интерпретировать OLS-оценки
# как несмещенные и перейти к тестированию гипотез при помощи Wald- и t-тестов, реализуем это при помощи функции:

def print_t(name, result):
    coef = result.effect[0]
    se = result.sd[0]
    z = result.tvalue[0]
    p = result.pvalue
    ci_low, ci_high = result.conf_int()[0]

    print(f"{name}: coef={coef:.4f}, p={p:.4g}, 95%CI=({ci_low:.4f}; {ci_high:.4f})")


test1 = ols.t_test("central = 0")       # H1

test2 = ols.t_test("middle_floor = 0")  # H2

test3a = ols.t_test("central_old = 0")  # H3 (часть 1)
test3b = ols.t_test("outsk_new = 0")    # H3 (часть 2)

print_t("H1 - Премия центра", test1)
print()
print_t("H2 - Средний этаж", test2)
print()
print_t("H3a - Ценовая значимость памятников архитектуры (старый фонд × центр)", test3a)
print_t("H3b - Ценовая значимость памятников архитектуры (новострой × окраина)", test3b)

H1 - Премия центра: coef=0.7277, p=1.643e-64, 95%CI=(0.6436; 0.8118)

H2 - Средний этаж: coef=0.0494, p=0.002796, 95%CI=(0.0170; 0.0818)

H3a - Ценовая значимость памятников архитектуры (старый фонд × центр): coef=-0.1809, p=0.3507, 95%CI=(-0.5607; 0.1990)
H3b - Ценовая значимость памятников архитектуры (новострой × окраина): coef=-0.0118, p=0.3959, 95%CI=(-0.0390; 0.0154)


**Выводы о протестированных гипотезах:**


***Гипотеза 1:*** *"Премия исторического центра" - подтверждена. Жилье в четырех «центральных» районах действительно дороже на примерно 73 %*

***Гипотеза 2:*** *"Средняя этажность - детерминант высокой стоимости жилья" - подтверждена. Недвижимость не на первых и в то же время не на последних этажах действительно имеет большую стоимость (2-й до (этаж_мах-1) +5 %)*

***Гипотеза 3:*** *"Историческая и ценовая значимость памятников архитектуры" - не подтверждена. У нас нет статистических оснований полагать, что в центральных районах города наибольшую ценность, которая выражается в более высокой стоимости, имеют дореволюционные дома, являющиеся памятниками архитектуры*